<a href="https://colab.research.google.com/github/lry0830/Smart-Cashierless-Checkout-System/blob/main/Image_Synthesis_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Extract Mask

In [ ]:
# Create folders for raw images
import os

if not os.path.exists('/content/data/Images'):
  os.makedirs('/content/data/Images')
else:
  print("Folder exists")

# Upload Mask Extraction script
from google.colab import files
%cd /content/data
uploaded = files.upload()

# Copy Raw Images to folder

# Upload Mask JSON file
from google.colab import files
%cd /content/data
uploaded = files.upload()

In [ ]:
%cd /content
# VGG Annotator : []()
!python Mask_Extraction_VGG.py "/content/data" "Mask.json"

In [ ]:
# Visualize Image & Mask
import matplotlib.pyplot as plt
import numpy as np

f, axarr = plt.subplots(1,2)
img = plt.imread("/content/data/Images/Boh/images/Boh.jpg")
mask = plt.imread("/content/data/Images/Boh/masks/Boh.png")
axarr[0].imshow(img)
axarr[1].imshow(mask, cmap='gray')

### 2. Image Augmentation

In [ ]:
# Install dependencies
!pip install bbaug
!pip install pascal_voc_writer
!pip install -U albumentations
!pip uninstall opencv
!pip install --upgrade opencv-python

In [ ]:
# Re-organize files
import shutil
import os

os.makedirs('/content/Extracted/images')
os.makedirs('/content/Extracted/masks')

for folder in os.listdir("/content/data/Images"):
  for x in os.listdir(os.path.join(f"/content/data/Images/{folder}", "images")):
    if x.endswith(".jpg"):
       shutil.copy(f"/content/data/Images/{folder}/images/{x}", f'/content/Extracted/images/{x}')
  for x in os.listdir(os.path.join(f"/content/data/Images/{folder}", "masks")):
    if x.endswith(".png"):
       shutil.copy(f"/content/data/Images/{folder}/masks/{x}", f'/content/Extracted/masks/{x}')


In [ ]:
# Upload annotation files
os.makedirs('/content/Extracted/txt')

In [ ]:
# For single-product images
%cd /content
!python Image_Augmentation_v4.py -N 3 -i '/content/Extracted/images' \
                                        -a '/content/Extracted/txt' \
                                        -m '/content/Extracted/masks' \
                                        -t syn

In [ ]:
# Check number of generated images
import os
import cv2

print(len(os.listdir('/content/augmented/images')))

img_name = (os.listdir('/content/augmented/images'))[0]
img = cv2.imread(('/content/augmented/images/'+img_name))
print(img.shape)

In [ ]:
import cv2
import matplotlib.pyplot as plt

img = cv2.imread('/content/augmented/images/Boh(3).jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
dh, dw, _ = img.shape

fl = open('/content/augmented/txt/Boh(3).txt', 'r')
data = fl.readlines()
fl.close()

for dt in data:
    dt = dt.replace("\n", '').split(' ')
    dt = dt[:5]

    # Split string to float
    _, x, y, w, h = map(float, dt)

    # Taken from https://github.com/pjreddie/darknet/blob/810d7f797bdb2f021dbe65d2524c2ff6b8ab5c8b/src/image.c#L283-L291
    # via https://stackoverflow.com/questions/44544471/how-to-get-the-coordinates-of-the-bounding-box-in-yolo-object-detection#comment102178409_44592380
    l = int((x - w / 2) * dw)
    r = int((x + w / 2) * dw)
    t = int((y - h / 2) * dh)
    b = int((y + h / 2) * dh)
    
    if l < 0:
        l = 0
    if r > dw - 1:
        r = dw - 1
    if t < 0:
        t = 0
    if b > dh - 1:
        b = dh - 1

    cv2.rectangle(img, (l, t), (r, b), (255, 0, 0), 5)

plt.imshow(img)
plt.show()

### 3. Train-Test Split
- `img2train.py`to generate train.txt
- `pickfile.py` to pick the images

In [ ]:
# Upload script
from google.colab import files
uploaded = files.upload()

In [ ]:
%cd /content
!python img2train.py "/content/augmented/images"

In [ ]:
# To split real images to train & test
!python pickfile_v2.py "/content/train.txt" \
                    "/content/augmented/images" \
                    "train"
                    
!python pickfile_v2.py "/content/val.txt" \
                    "/content/augmented/images" \
                    "val"

In [ ]:
# Convert to json (Require classes.txt)
%cd /content
!python yolov5_2_coco.py --dir_path "/content/augmented"

In [ ]:
# Visualize 
!python coco_visual.py --vis_num 10 \
                       --json_path /content/augmented_COCO_format/annotations/instances_train2017.json \
                       --img_dir /content/augmented_COCO_format/train2017

img = plt.imread("/content/visualized.jpg")
plt.imshow(img)

### 4. Export to Google Drive

In [ ]:
# Download Augmented Synthesized Images
from google.colab import files

!zip -r /content/augmented_COCO_raw.zip /content/augmented_COCO_format
files.download('/content/augmented_COCO_raw.zip')

### 5. Image Synthesis

In [ ]:
# Add bg.jpg to /content

# Image Synthesis
!python synthesis.py --count 1 --local_rank 1

In [ ]:
!python coco_visual.py --vis_num 15 \
                       --json_path /content/annotations.json \
                       --img_dir /content/synthesized_dataset

img = plt.imread('/content/visualized.jpg')
plt.imshow(img)

### 7. CycleGAN Training

In [ ]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

In [ ]:
!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

In [ ]:
%cd /content
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix.git

In [ ]:
%cd /content/pytorch-CycleGAN-and-pix2pix
!pip install -r requirements.txt

In [ ]:
!unzip /content/drive/MyDrive/Data_Synthesis/CycleGAN_v7.zip -d /content/dataset

In [ ]:
!python train.py --dataroot /content/dataset --name syn_2_real --model cycle_gan \
                 --netD  basic --n_epochs 100 --n_epochs_decay 100 --batch_size 4\
                  --netG resnet_9blocks --preprocess scale_width_and_crop --load_size 900 \
                  --crop_size 256 --display_id 0 --lambda_identity 0.4 --lambda_A 8 --lambda_B 8

In [ ]:
# Place in pytorch-CycleGAN-and-pix2pix/checkpoints
!cp /content/drive/MyDrive/CycleGAN_Weights/latest_net_G.pth /content/pytorch-CycleGAN-and-pix2pix/checkpoints/syn_2_real
#os.rename('/content/pytorch-CycleGAN-and-pix2pix/checkpoints/syn_2_real/latest_net_G_A.pth', 
          #'/content/pytorch-CycleGAN-and-pix2pix/checkpoints/syn_2_real/latest_net_G.pth')

### 8. CycleGAN Rendering

In [ ]:
!unzip /content/drive/MyDrive/Data_Synthesis/CycleGAN_v7.zip -d /content/dataset

In [ ]:
!python test.py --dataroot /content/dataset/ \
                --direction AtoB --model test \
                --name syn_2_real --preprocess scale_width \
                --load_size 800 --no_flip --netG resnet_6blocks \
                --no_dropout --num_test 10

### Extra : Miscellaneous Codes

In [ ]:
import shutil
shutil.rmtree('/content/dataset')